# Outline

- Bayesian inference recap: What? Why? How?
- What is a Bayesian workflow and why do we need it?
- Principles of Bayesian workflow
- Modern Bayesian workflow

# Bayesian inference

## Bayes formula:

 $$p(\theta|y) = \frac{p(y | \theta) p(\theta)}{p(y)}$$
Can you recall what the components of the Bayes formula are?

![](assets/bmeh_normal.png){.r-stretch}

## Bayes formula:

 $$p(\theta|y) = \frac{p(y | \theta) p(\theta)}{p(y)}$$
Can you recall what the components of the Bayes formula are?

![](assets/bmeh_normal.png){.r-stretch}

- $p(\theta)$ is the *prior* distribution, i.e. what is known *a priori*
- $p(y|\theta)$ is the *likelihood*, i.e. probability of observing the data given parameters $\theta$
- $p(\theta|y)$ is the *posterior* distribution, i.e. the distribution of parameters of interest after data were observed


## Bayes rule:

$$\underbrace{p(\theta|y)}_\text{posterior} \propto \underbrace{p(y | \theta)}_{\text{likelihood}}  \underbrace{p(\theta)}_{\text{prior}}$$

. . .


What can possibly go wrong?

. . .

A lot can go wrong!


## General principle of Bayesian inference:

- Specify a complete Bayesian model:
  - specify likelihood
  - specify priors for all parameters (saying "I don't know" is also a prior)
    
## General principle of Bayesian inference:

- Example:
  - consider <ins>data</ins> $y = \{ y_1, ..., y_n\}$
  - specify an <ins>observation model</ins>, e.g. $$p(y|\theta) = \prod_i N(y_i | \theta, \sigma^2)$$
   <!-- - here $\theta$ is a parameter which we want to infer-->
  - complete the model with a prior distribution, e.g. $$p(\theta)=N(0,1)$$


## General principle of Bayesian inference:

- Specify a complete Bayesian model
- Sample the posterior distribution of the parameter $\theta$.
  
. . .

Sometimes posterior is available in a closed form. 

. . .

But rarely.

## Probabilistic programming languages

Probabilistic programming languages (PPLs) from a user's perspective:

- PPLs are designed to let the user <ins>focus on modelling</ins> while inference happens automatically.
- Users need to specify
  - prior,
  - likelihood.
- Inference is performed via powerful algorithms such as, for example, Markov Chain Monte Carlo (<ins>MCMC</ins>).
- Availability of <ins>diagnostic tools</ins>.

## Remark on inference methods

- Alongside "exact"methods such as MCMC, there also exist approximate methods, such as Variation Inference (VI). 
- We are opting to use MCMC whenever possible since it has theoretical guarantees.


## Diagnosing MCMC outputs

We use multiple chains of MCMC to estimate the posterior:

. . .

<center>
![](assets/trace_theta.png){width=50%}



## Diagnosing MCMC outputs

- <ins>Convergence diagnostics</ins>
  - $\hat{R}$ statistic,
  - traceplots.
- <ins>Effective sample size (ESS)</ins>:
  - samples will be typically autocorrelated within a chain, which increases the uncertainty of the estimation of posterior quantities,
  - ESS -- number of *independent* samples required to obtain the same level of uncertainty as from the available dependent samples.

## Diagnosing MCMC outputs

We use multiple chains of MCMC to inspect convergence after warm-up:

. . .

<center>
![](assets/trace_theta.png){width=50%} 



## Diagnosing MCMC outputs

We use multiple chains and inspect convergence after warm-up:

. . .

<center>
![](assets/trace_theta_bad.png){width=50%} 



## Diagnosing MCMC outputs

The post-warm-up samples of $\theta$ approximate its posterior distribution:

. . .

<center>
![](assets/post_theta.png){width=50%}

# Principles of Bayesian workflow

## Workflows as a 'good practice'

Workflows exist in a variety of disciplines. For example, in machine learning workflow standards are being formalised under the name of MLOps:

<center>
![](assets/mlops-loop-en.png){width=50%}


## Box's loop

In the 1960's, the statistician Box formulated the notion of a loop to understand the nature of the scientific method. This loop is called Box's loop by Blei et. al. (2014):

<center>
![](assets/boxes_loop.png){width=60%} 

## Modern Bayesian workflow

A systematic review of the steps within the modern Bayesian workflow, described in Gelman et al. (2020):

<center>
![](assets/bayes_workflow.png){width=40%} 

## Prior predictive checks

<ins>Prior predictive checking</ins> consists in simulating data from the priors:

- visualize priors (especially after transformation),

- this shows the range of data compatible with the model,

- it helps understand the adequacy of the chosen priors, as it is often easier to elicit expert knowledge on measureable quantities of interest rather than abstract parameter values.


## Iterative model building

A possible realisation of the Bayeisan workflow loop:

- Understand the <ins>domain</ins> and problem,

- Formulate the model <ins>mathematically</ins>,

- Implement model, test, <ins>debug</ins>,

- <ins>debug, debug, debug</ins>


## Iterative model building

- Understand the <ins>domain</ins> and problem,

- Formulate the model <ins>mathematically</ins>,

- Implement model, test, <ins>debug</ins>,

- Perform <ins>prior predictive</ins> check,

- Fit the model,

- Assess <ins>convergence diagnostics</ins>,

- Perform <ins>posterior predictive</ins> check,

- Improve the model <ins>iteratively</ins>: from baseline to complex and computationally efficient models.

# Examples

## Data
Assume that the true data comes from the model
$$y_i = a + b x_i + \epsilon_i, \quad \epsilon_i \sim N(0, \sigma^2).$$

<center>
![](assets/data1.png){width=45%} 



## Model
We implemented the model in our favorite PPL:

. . .


In [ ]:
data {
  int<lower=0> N;
  vector[N] y;
  vector[N] x;
  real<lower=0> sigma;
}
parameters {
  real a;
  real b;
}
model {
  y ~ normal(a, sigma);
}

## Prior predictive check
Let us draw from the priors, i.e. we are not using any data at this stage, only trying to see what kind of data ($y$) this model is able to generate.

. . .

<center>
![](assets/data2.png){width=45%} 

(add uncertainty bounds)

## Prior predictive check
Something doesn't look right...

. . .

<center>
![](assets/data2.png){width=45%} 


## Debug
There was a bug in the model. Let's correct it:

```
data {
  int<lower=0> N;
  vector[N] y;
  vector[N] x;
  real<lower=0> sigma;
}
parameters {
  real a ~ normal(0,1);
  real b ~ normal(0,1);
}
model {
  y ~ normal(a + b*x, sigma);
}
```

## Prior redictive check again
<center>
![](assets/data3.png){width=45%} 

Better: now the range of prior predictive distribution is covering the data.

(add uncertainty bounds)


## Fit the model
Print summary:
<center>
![](assets/fit1.png){width=60%} 

## Plot estimates
<center>
![](assets/est1.png){width=45%} 


## Convergence diagnostics


<center>
![](assets/fit1_conv.png){width=60%} 


## Convergence diagnostics

Traceplots

<center>
![](assets/traceplots1.png){width=45%} 

## Posterior predictive check

<center>
![](assets/posterior1.png){width=45%} 

(add uncertainty bounds)

# References